## 1. 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")

라이브러리 로드 완료


## 2. 데이터 로드

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
holidays = pd.read_csv('holidays_events.csv')
oil = pd.read_csv('oil.csv')
transactions = pd.read_csv('transactions.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print("\n데이터 로드 완료")

Train shape: (3000888, 6)
Test shape: (28512, 5)

데이터 로드 완료


## 3. 날짜 파싱

In [3]:
# 날짜를 datetime 형식으로 변환
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays['date'] = pd.to_datetime(holidays['date'])
transactions['date'] = pd.to_datetime(transactions['date'])

print(f"Train 기간: {train['date'].min()} ~ {train['date'].max()}")
print(f"Test 기간: {test['date'].min()} ~ {test['date'].max()}")

Train 기간: 2013-01-01 00:00:00 ~ 2017-08-15 00:00:00
Test 기간: 2017-08-16 00:00:00 ~ 2017-08-31 00:00:00


## 4. 날짜 기반 특성 추출

In [4]:
def extract_date_features(df):
    """날짜에서 여러 특성 추출"""
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['quarter'] = df['date'].dt.quarter
    df['dayofweek'] = df['date'].dt.dayofweek  # 0=월요일, 6=일요일
    df['week'] = df['date'].dt.isocalendar().week
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['date'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['date'].dt.is_quarter_end.astype(int)
    
    # ✓ 추가: Cyclical 특성 (삼각함수 변환)
    # month와 dayofweek는 순환형이므로 sin/cos로 변환해야 함
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    return df

# Train, Test에 특성 추출
train = extract_date_features(train)
test = extract_date_features(test)

print("\n추출된 날짜 특성:")
print(train[['date', 'year', 'month', 'day', 'dayofweek', 'month_sin', 'month_cos', 'dayofweek_sin']].head())


추출된 날짜 특성:
        date  year  month  day  dayofweek  month_sin  month_cos  dayofweek_sin
0 2013-01-01  2013      1    1          1        0.5   0.866025       0.781831
1 2013-01-01  2013      1    1          1        0.5   0.866025       0.781831
2 2013-01-01  2013      1    1          1        0.5   0.866025       0.781831
3 2013-01-01  2013      1    1          1        0.5   0.866025       0.781831
4 2013-01-01  2013      1    1          1        0.5   0.866025       0.781831


## 5. Stores 정보 병합

In [5]:
# Store 정보를 Train, Test에 병합
train = train.merge(stores, on='store_nbr', how='left')
test = test.merge(stores, on='store_nbr', how='left')

print(f"Train shape after merge: {train.shape}")
print(f"Train columns: {list(train.columns)}")
print("\nStore 병합 완료")

Train shape after merge: (3000888, 24)
Train columns: ['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'year', 'month', 'day', 'quarter', 'dayofweek', 'week', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos', 'city', 'state', 'type', 'cluster']

Store 병합 완료


## 6. Oil 정보 병합

In [6]:
# Oil 가격 정보 병합
oil_renamed = oil.rename(columns={'dcoilwtico': 'oil_price'})

train = train.merge(oil_renamed, on='date', how='left')
test = test.merge(oil_renamed, on='date', how='left')

print(f"결측치 (train): {train['oil_price'].isnull().sum()}")
print(f"결측치 (test): {test['oil_price'].isnull().sum()}")

# 결측치 처리: 선형 보간
train['oil_price'] = train['oil_price'].interpolate(method='linear')
test['oil_price'] = test['oil_price'].interpolate(method='linear')

# 여전히 남은 결측치는 평균값으로 채우기
train['oil_price'].fillna(train['oil_price'].mean(), inplace=True)
test['oil_price'].fillna(test['oil_price'].mean(), inplace=True)

print(f"\n처리 후 결측치 (train): {train['oil_price'].isnull().sum()}")
print(f"처리 후 결측치 (test): {test['oil_price'].isnull().sum()}")

결측치 (train): 928422
결측치 (test): 7128

처리 후 결측치 (train): 0
처리 후 결측치 (test): 0


## 7. Holidays/Events 정보 병합

In [7]:
# 휴일 정보 - 국가 휴일과 지역 휴일로 분리
holidays_national = holidays[holidays['locale'] == 'National'][['date', 'type']].copy()
holidays_national.rename(columns={'type': 'holiday_type'}, inplace=True)
holidays_national['is_holiday'] = 1

# Train, Test에 국가 휴일 정보 병합
train = train.merge(holidays_national[['date', 'is_holiday']], on='date', how='left')
test = test.merge(holidays_national[['date', 'is_holiday']], on='date', how='left')

# 결측치는 0 (휴일 아님)
train['is_holiday'].fillna(0, inplace=True)
test['is_holiday'].fillna(0, inplace=True)

print(f"Train 휴일 비율: {train['is_holiday'].sum() / len(train) * 100:.2f}%")
print(f"Test 휴일 비율: {test['is_holiday'].sum() / len(test) * 100:.2f}%")

Train 휴일 비율: 8.71%
Test 휴일 비율: 0.00%


## 8. Transactions 정보 병합

In [8]:
# 거래량 정보 병합
transactions_renamed = transactions.rename(columns={'transactions': 'n_transactions'})

train = train.merge(transactions_renamed, on=['date', 'store_nbr'], how='left')
test = test.merge(transactions_renamed, on=['date', 'store_nbr'], how='left')

print(f"Train 거래량 결측치: {train['n_transactions'].isnull().sum()}")
print(f"Test 거래량 결측치: {test['n_transactions'].isnull().sum()}")

# 결측치 처리: 각 가게별 평균 거래량으로 채우기
store_avg_transactions = train.groupby('store_nbr')['n_transactions'].mean()
train['n_transactions'].fillna(train['store_nbr'].map(store_avg_transactions), inplace=True)
test['n_transactions'].fillna(test['store_nbr'].map(store_avg_transactions), inplace=True)

print(f"\n처리 후 Train 거래량 결측치: {train['n_transactions'].isnull().sum()}")
print(f"처리 후 Test 거래량 결측치: {test['n_transactions'].isnull().sum()}")

Train 거래량 결측치: 246081
Test 거래량 결측치: 28512

처리 후 Train 거래량 결측치: 0
처리 후 Test 거래량 결측치: 0


In [ ]:
def create_yoy_features(df, target_col='sales'):
    """
    Year-over-Year 특성: 365일 전 판매량 (같은 계절)
    """
    df = df.sort_values(['store_nbr', 'date']).reset_index(drop=True)
    
    # 365일 전 판매량
    df[f'{target_col}_yoy_365'] = df.groupby('store_nbr')[target_col].shift(365)
    
    return df

# Train에 YoY features 추가
print("Year-over-Year features 생성 중...")
train = create_yoy_features(train, target_col='sales')
train = create_yoy_features(train, target_col='n_transactions')

print("✓ YoY features 추가 완료")
print(f"추가된 특성: sales_yoy_365, n_transactions_yoy_365")

In [ ]:
# Test 데이터도 같은 방식으로 처리하되, Train의 마지막 데이터를 참고해야 함
print("Test 데이터에 시계열 특성 추가 중...")

# Train과 Test를 병합해서 lag 계산 (시간 연속성 보장)
train_test_combined = pd.concat([train[['store_nbr', 'date', 'sales', 'n_transactions']], 
                                 test[['store_nbr', 'date', 'n_transactions']]], 
                                axis=0, ignore_index=True)

# 병합 데이터에서 Lag/Rolling 특성 생성
train_test_combined = create_lag_features(train_test_combined, target_col='sales', lags=[1, 7, 14, 30])
train_test_combined = create_lag_features(train_test_combined, target_col='n_transactions', lags=[1, 7, 14, 30])
train_test_combined = create_rolling_features(train_test_combined, target_col='sales', windows=[7, 14, 30])
train_test_combined = create_rolling_features(train_test_combined, target_col='n_transactions', windows=[7, 14, 30])
train_test_combined = create_yoy_features(train_test_combined, target_col='sales')
train_test_combined = create_yoy_features(train_test_combined, target_col='n_transactions')

# Test 부분만 추출
test_lag_features = train_test_combined[train_test_combined.index >= len(train)].copy()
test_lag_features = test_lag_features.drop(columns=['sales', 'n_transactions'], errors='ignore')

# Test에 원래 n_transactions 다시 추가
for col in test_lag_features.columns:
    if col not in test.columns and col != 'date' and col != 'store_nbr':
        test[col] = test_lag_features[col].values

print("✓ Test 데이터 시계열 특성 추가 완료")

# 결측치 처리 (처음 몇 행들은 lag가 없을 수 있음)
lag_cols = [col for col in train.columns if 'lag' in col or 'roll' in col or 'yoy' in col]
print(f"\n추가된 총 {len(lag_cols)}개 특성의 결측치 처리...")

# 결측치를 각 가게별 평균값으로 채우기
for col in lag_cols:
    if col in train.columns:
        fill_value = train.groupby('store_nbr')[col].transform('mean')
        train[col].fillna(fill_value, inplace=True)
        train[col].fillna(train[col].mean(), inplace=True)
    
    if col in test.columns:
        fill_value = test.groupby('store_nbr')[col].transform('mean')
        test[col].fillna(fill_value, inplace=True)
        test[col].fillna(test[col].mean(), inplace=True)

print("✓ 결측치 처리 완료")

## 12. Test 데이터에 Lag/Rolling 특성 추가

## 11. Year-over-Year 특성 (작년 같은 날짜의 판매량)

In [ ]:
def create_rolling_features(df, target_col='sales', windows=[7, 14, 30]):
    """
    Rolling statistics 생성: 이동평균, 표준편차, 최소/최대값
    """
    df = df.sort_values(['store_nbr', 'date']).reset_index(drop=True)
    
    for window in windows:
        # 이동평균
        df[f'{target_col}_roll_mean_{window}'] = df.groupby('store_nbr')[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        # 이동 표준편차
        df[f'{target_col}_roll_std_{window}'] = df.groupby('store_nbr')[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
        # 최소값
        df[f'{target_col}_roll_min_{window}'] = df.groupby('store_nbr')[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).min()
        )
        # 최대값
        df[f'{target_col}_roll_max_{window}'] = df.groupby('store_nbr')[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).max()
        )
    
    return df

# Train에 Rolling features 추가
print("Rolling features 생성 중...")
train = create_rolling_features(train, target_col='sales', windows=[7, 14, 30])
train = create_rolling_features(train, target_col='n_transactions', windows=[7, 14, 30])

print("✓ Rolling features 추가 완료")
print(f"추가된 특성: sales/n_transactions의 7/14/30일 이동평균, 표준편차, 최소/최대값")

## 10. Rolling 특성 추출 (이동 평균 및 표준편차)

In [ ]:
def create_lag_features(df, target_col='sales', lags=[1, 7, 14, 30]):
    """
    Lag features 생성: 이전 날짜들의 판매량
    각 store별로 따로 계산해야 시간순서 유지됨
    """
    df = df.sort_values(['store_nbr', 'date']).reset_index(drop=True)
    
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df.groupby('store_nbr')[target_col].shift(lag)
    
    return df

# Train에 Lag features 추가 (test는 나중에 처리)
print("Lag features 생성 중...")
train = create_lag_features(train, target_col='sales', lags=[1, 7, 14, 30])
train = create_lag_features(train, target_col='n_transactions', lags=[1, 7, 14, 30])

print("✓ Lag features 추가 완료")
print(f"Train에 추가된 특성: sales_lag_1, sales_lag_7, sales_lag_14, sales_lag_30")
print(f"                    n_transactions_lag_1, n_transactions_lag_7, n_transactions_lag_14, n_transactions_lag_30")

## 9. Lag 특성 추출 (시계열 이전 데이터)

## 13. Categorical 특성 인코딩

In [9]:
# 카테고리 특성 확인
categorical_cols = ['type', 'family']

print("카테고리 값 개수:")
for col in categorical_cols:
    print(f"{col}: {train[col].nunique()}")

# One-hot encoding
train = pd.get_dummies(train, columns=categorical_cols, drop_first=False)
test = pd.get_dummies(test, columns=categorical_cols, drop_first=False)

# Train과 Test의 컬럼 맞추기
train_cols = set(train.columns)
test_cols = set(test.columns)

# Test에만 있는 컬럼은 제거
for col in test_cols - train_cols:
    test.drop(col, axis=1, inplace=True)

# Train에만 있는 컬럼은 Test에 0으로 추가
for col in train_cols - test_cols:
    test[col] = 0

# 컬럼 순서 맞추기
test = test[train.columns.drop('sales', errors='ignore')]

print(f"\nTrain shape: {train.shape}")
print(f"Test shape: {test.shape}")

카테고리 값 개수:
type: 5
family: 33

Train shape: (3008016, 63)
Test shape: (28512, 62)


## 14. 결측치 최종 확인

In [10]:
print("=== Train 결측치 ===")
missing_train = train.isnull().sum()
print(missing_train[missing_train > 0])

print("\n=== Test 결측치 ===")
missing_test = test.isnull().sum()
print(missing_test[missing_test > 0])

# 결측치가 없으면
if missing_train.sum() == 0:
    print("\n✓ Train에 결측치 없음")
if missing_test.sum() == 0:
    print("✓ Test에 결측치 없음")

=== Train 결측치 ===
Series([], dtype: int64)

=== Test 결측치 ===
Series([], dtype: int64)

✓ Train에 결측치 없음
✓ Test에 결측치 없음


## 15. 데이터 통계

In [11]:
print("\n=== Train 데이터 정보 ===")
print(f"Shape: {train.shape}")
print(f"컬럼 수: {len(train.columns)}")
print(f"\n수치형 특성 통계:")
print(train.describe())


=== Train 데이터 정보 ===
Shape: (3008016, 63)
컬럼 수: 63

수치형 특성 통계:
                 id                           date     store_nbr  \
count  3.008016e+06                        3008016  3.008016e+06   
mean   1.501508e+06  2015-04-24 22:50:02.843602688  2.750000e+01   
min    0.000000e+00            2013-01-01 00:00:00  1.000000e+00   
25%    7.520038e+05            2014-02-27 18:00:00  1.400000e+01   
50%    1.502226e+06            2015-04-25 12:00:00  2.750000e+01   
75%    2.248883e+06            2016-06-18 06:00:00  4.100000e+01   
max    3.000887e+06            2017-08-15 00:00:00  5.400000e+01   
std    8.657303e+05                            NaN  1.558579e+01   

              sales   onpromotion          year         month           day  \
count  3.008016e+06  3.008016e+06  3.008016e+06  3.008016e+06  3.008016e+06   
mean   3.582691e+02  2.609620e+00  2.014839e+03  6.209123e+00  1.561789e+01   
min    0.000000e+00  0.000000e+00  2.013000e+03  1.000000e+00  1.000000e+00   
25%    

## 16. 전처리된 데이터 저장

In [12]:
# 전처리된 데이터 저장
train.to_csv('train_processed.csv', index=False)
test.to_csv('test_processed.csv', index=False)

print("✓ train_processed.csv 저장 완료")
print("✓ test_processed.csv 저장 완료")
print(f"\n총 {len(train)} 행의 학습 데이터 준비 완료")
print(f"총 {len(test)} 행의 테스트 데이터 준비 완료")

✓ train_processed.csv 저장 완료
✓ test_processed.csv 저장 완료

총 3008016 행의 학습 데이터 준비 완료
총 28512 행의 테스트 데이터 준비 완료


## 17. 특성 목록 확인

In [13]:
# Train에서 target 제외한 특성 목록
feature_cols = [col for col in train.columns if col not in ['sales', 'date', 'id']]

print(f"\n총 특성 개수: {len(feature_cols)}")
print("\n특성 목록:")
for i, col in enumerate(feature_cols, 1):
    print(f"{i}. {col}")


총 특성 개수: 60

특성 목록:
1. store_nbr
2. onpromotion
3. year
4. month
5. day
6. quarter
7. dayofweek
8. week
9. is_month_start
10. is_month_end
11. is_quarter_start
12. is_quarter_end
13. month_sin
14. month_cos
15. dayofweek_sin
16. dayofweek_cos
17. city
18. state
19. cluster
20. oil_price
21. is_holiday
22. n_transactions
23. type_A
24. type_B
25. type_C
26. type_D
27. type_E
28. family_AUTOMOTIVE
29. family_BABY CARE
30. family_BEAUTY
31. family_BEVERAGES
32. family_BOOKS
33. family_BREAD/BAKERY
34. family_CELEBRATION
35. family_CLEANING
36. family_DAIRY
37. family_DELI
38. family_EGGS
39. family_FROZEN FOODS
40. family_GROCERY I
41. family_GROCERY II
42. family_HARDWARE
43. family_HOME AND KITCHEN I
44. family_HOME AND KITCHEN II
45. family_HOME APPLIANCES
46. family_HOME CARE
47. family_LADIESWEAR
48. family_LAWN AND GARDEN
49. family_LINGERIE
50. family_LIQUOR,WINE,BEER
51. family_MAGAZINES
52. family_MEATS
53. family_PERSONAL CARE
54. family_PET SUPPLIES
55. family_PLAYERS AND ELEC